<a href="https://colab.research.google.com/github/DilshanManathunge/TensorflowPractice/blob/master/NLP/movie_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf 
print(tf.__version__)

2.2.0


In [3]:
import tensorflow_datasets as tfds 
imdb,info = tfds.load("imdb_reviews",with_info=True,as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteGLSCNH/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteGLSCNH/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteGLSCNH/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np 
train_data,test_data = imdb['train'],imdb['test']

In [9]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy().decode('utf8')))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(str(s.numpy().decode('utf8')))
  testing_labels.append(l.numpy())

In [10]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [12]:
vocab_size = 10000
embedding_dims = 16
max_length = 120
trunc_type = 'post'
ovv_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [13]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token = ovv_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index 
print(len(word_index))

88583


In [18]:
sequences = tokenizer.texts_to_sequences(training_sentences)

In [19]:
padded = pad_sequences(sequences,maxlen=max_length,padding=trunc_type)

In [21]:
testing_sequneces = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequneces,maxlen=max_length)

In [22]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

In [23]:
def review_decode(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text]) 

In [25]:
print(review_decode(padded[3]))
print(testing_sentences[3])

this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Poor Shirley MacLaine tries hard to lend some gravitas to this mawkish, gag-inducing "feel-good" movie, but she's trampled by the run-away sentimentality of a film that's not the least bit grounded in reality.<br /><br />This was directed by Curtis Hanson? Did he have a lobotomy since we last heard from him? Hanson can do effective drama sprinkled with comedy, as evidenced by "Wonder Boys." So I don't know what happened to him here. This is the kind of movie that doesn't want to ac

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Embedding

In [29]:
model = Sequential([
                    Embedding(vocab_size,embedding_dims,input_length=max_length),
                    Flatten(),
                    Dense(units = 6,activation ='relu'),
                    Dense(units = 1 , activation = "sigmoid")


])
model.compile(loss="binary_crossentropy",optimizer = 'adam',metrics= ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(padded,training_labels_final,epochs=10,validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4751 - accuracy: 0.7507 - val_loss: 0.3206 - val_accuracy: 0.8615
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2197 - accuracy: 0.9145 - val_loss: 0.3339 - val_accuracy: 0.8572
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.1004 - accuracy: 0.9726 - val_loss: 0.4070 - val_accuracy: 0.8453
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0338 - accuracy: 0.9944 - val_loss: 0.4814 - val_accuracy: 0.8453
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0136 - accuracy: 0.9981 - val_loss: 0.5244 - val_accuracy: 0.8460
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0058 - accuracy: 0.9991 - val_loss: 0.5979 - val_accuracy: 0.8395
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0018 - accuracy: 0.9999 - val_loss: 0.6518 - val_accuracy: 0.8392
Epoch 

In [32]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [33]:
import io

out_v = io.open('ves.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1,vocab_size):
  word = reverse_word_index[word_num]
  embbedings  = weights[word_num]
  out_m.write(word + '\n')
  out_v.write('\t'.join([str(x) for x in embbedings])+ "\n")
out_m.close()
out_v.close()


In [36]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('ves.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
